<img src="https://github.com/seap-udea/xearch/blob/main/resources/xearch-logo.png?raw=true" align="left">

## Required packages

In [1]:
# External modules
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as c
from astropy.time import Time
import pandas as pd
import pickle
from datetime import datetime
from tqdm import tqdm
from IPython.display import display,HTML

# Constants
G = c.G.value
Msun = c.M_sun.value
Mearth = c.M_earth.value
Mjup = c.M_jup.value
Rsun = c.R_sun.value
Rearth = c.R_earth.value
Rjup = c.R_jup.value
Mjup2earth = Mjup/Mearth
Rjup2earth = Rjup/Rearth

# Units
K = 1 # Kelvin
days = 86400
JD = days # Julian days
BJD = days # Bessel Julian days
years = 365.25*days # Julian years
au = c.au.value 
deg = np.pi/180
rad = 1/deg
Gyr = 1e9*years
dex = 1

# Useful routines
def sex2dec(angle):
    h, m, s = map(float, angle.split())
    sgn = np.sign(h)
    return sgn*(abs(h) + m/60 + s/3600)

def date2jd(date, format='%y/%m/%d'):
    dt = datetime.strptime(date, format)
    t = Time(dt)
    jd = t.jd
    return jd

def print_df(df):
    display(HTML(df.to_html()))


## Read database

In [60]:
import xml.etree.ElementTree as ET, urllib.request, gzip, io
#url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems.xml.gz";suffix='open'
url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems-exoplanetarchive.xml.gz";suffix='nasa'
#url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems-exoplaneteu.xml.gz";suffix='eu'

oec = ET.parse(gzip.GzipFile(fileobj=io.BytesIO(urllib.request.urlopen(url).read())))

In [61]:
# Output mass and radius of all planets 
for planet in oec.findall(".//planet"):
    name = planet.findtext("name") 
    if 'Proxima' in name:
        print(name)
        print(planet.findtext("mass"))
    pass

Proxima Cen b
0.00337


In [62]:
binary_properties = dict(
    # General
    name = ['binary_name',str,0],
    # Position
    separation = ['sep_bin',float,1,au], 
    positionangle = ['pa_bin',float,1,deg],
    # Orbital elements
    semimajoraxis = ['a_bin',float,1],
    eccentricity = ['e_bin',float,1],
    inclination = ['I_bin',float,1],
    periastron = ['w_bin',float,1,deg],
    ascendingnode = ['W_bin',float,1,deg],
    meananomaly = ['M_bin',float,1],
    meanlongitude = ['L_bin',float,1,deg],
    period = ['period_bin',float,1,days],
    epoch = ['et_bin',float,1,BJD],
    # Times
    maximumrvtime = ['tk_bin',float,1,BJD],
    # Magnitudes
    magB = ['magB_bin',float,1,dex],
    magV = ['magV_bin',float,1,dex],
    magI = ['magI_bin',float,1,dex],
    magR = ['magR_bin',float,1,dex],
    magJ = ['magJ_bin',float,1,dex],
    magH = ['magH_bin',float,1,dex],
    magK = ['magK_bin',float,1,dex],
) # Other: nstars

system_properties = dict(
    # General
    name = ['system_name',str,0],
    # Position
    declination = ['declination',sex2dec,1,deg],
    rightascension = ['rightascension',sex2dec,1,deg],
    distance = ['distance',float,1],
) # Other: nplanets

star_properties = dict(
    # General
    name = ['star_name',str,0],
    # Physical
    mass = ['star_mass',float,1,Msun],
    radius = ['star_radius',float,1,Rsun],   
    temperature = ['star_temperature',float,1,K],
    age = ['star_age',float,1,Gyr],
    # Spectral
    metallicty = ['metallicity',float,1,dex], # Relative to sun
    spectraltype = ['spectraltype',str,0],
    # Magnitudes
    magB = ['magB',float,1,dex],
    magV = ['magV',float,1,dex],
    magI = ['magI',float,1,dex],
    magR = ['magR',float,1,dex],
    magJ = ['magJ',float,1,dex],
    magH = ['magH',float,1,dex],
    magK = ['magK',float,1,dex],
)

planet_properties = dict(
    # General
    name = ['planet_name',str,0],
    # description = ['description',str,0],
    discoverymethod = ['method',str,0],
    # Times
    istransiting = ['istransiting',int,1],
    discoveryyear = ['dicovery',int,1,years],
    #lastupdate = ['update',date2jd,1,JD],
    lastupdate = ['update',str,0],
    # Physical    
    mass = ['mass',float,Mjup2earth,Mjup],
    radius = ['radius',float,Rjup2earth,Rjup],
    temperature = ['temperature',float,1,K],
    age = ['age',float,1,Gyr],
    # Position
    impactparameter = ['b',float,1],
    separation = ['separation',float,1,au],
    # Orbital elements
    semimajoraxis = ['a',float,1],
    eccentricity = ['e',float,1],
    inclination = ['I',float,1],
    periastron = ['w',float,1,deg],
    ascendingnode = ['W',float,1,deg],
    meananomaly = ['M',float,1],
    meanlongitude = ['L',float,1,deg],
    period = ['period',float,1,days],
    epoch = ['et',float,1,BJD],
    # Times
    transittime = ['tt',float,1,BJD],
    periastrontime = ['tp',float,1],
    maximumrvtime = ['tk',float,1,BJD],
    # Other
    spinorbitalignment = ['so',float,1,deg],
)
# Columns compilation
columns = []
for properties in system_properties,star_properties,planet_properties: #,binary_properties:
    for key,item in properties.items():
        columns.append(item[0])
print(f"Number of columns: {len(columns)}")

Number of columns: 42


Getting data:

In [63]:
nsystems = 0
nallplanets = 0 
systems = dict()

exoplanets_list = []
for system in tqdm(oec.findall('.//system')):
    name = system.findtext('name')
    if name is None:
        continue
    nsystems += 1
    
    systems[name] = dict()
    exoplanet = dict()

    system_dict = dict()
    # System properties
    for key,item in system_properties.items():
        value = system.findtext(key)
        if (value is not None) and (value != ''):
            finalvalue = item[1](value)
            if item[1] != str:
                finalvalue *= item[2]
        else:
            finalvalue = None
        systems[name][item[0]] = finalvalue
        system_dict[item[0]] = finalvalue

    # Stars
    nstars = 0
    systems[name]['stars'] = []
    for star in system.findall('.//star'):
        star_name = star.findtext('name') 
        if star_name is None:
            continue
        nstars += 1
        star_dict = dict()
        for key,item in star_properties.items():
            value = star.findtext(key)
            if (value is not None) and (value != ''):
                finalvalue = item[1](value)
                if item[1] != str:
                    finalvalue *= item[2]
            else:
                finalvalue = None

            star_dict[item[0]] = finalvalue
        systems[name]['stars'].append(star_dict)
        system_dict.update(star_dict)

    systems[name]['nstars'] = nstars
    
    # Planets
    nplanets = 0
    systems[name]['planets'] = []
    for planet in system.findall('.//planet'):
        planet_name = planet.findtext('name') 
        if planet_name is None:
            continue
        nallplanets += 1
        nplanets += 1
        planet_dict = dict()
        for key,item in planet_properties.items():
            value = planet.findtext(key)
            if (value is not None) and (value != ''):
                finalvalue = item[1](value)
                if item[1] != str:
                    finalvalue *= item[2]
            else:
                finalvalue = None

            planet_dict[item[0]] = finalvalue
            exoplanet[item[0]] = finalvalue

        systems[name]['planets'].append(planet_dict)
        system_dict_planet = system_dict.copy()
        system_dict_planet.update(planet_dict)

        exoplanets_list.append(system_dict_planet)

    systems[name]['nplanets'] = nplanets
    exoplanet['nplanets'] = nplanets

exoplanets = pd.DataFrame(exoplanets_list)
for names in ['system_name','star_name','planet_name']:
    exoplanets[names] = exoplanets[names].str.replace('&ouml;', 'o') # Fixing some encoding issues

# Save dictionary data
with open(f"systems-{suffix}.pickle", 'wb') as file:
    pickle.dump(systems, file)

# Save dataframe
exoplanets.to_csv(f"exoplanets-{suffix}.csv",index=False)

print_df(exoplanets.describe())

100%|██████████| 4164/4164 [00:00<00:00, 24729.98it/s]


,declination,rightascension,distance,star_mass,star_radius,star_temperature,star_age,magV,magI,magJ,magH,magK,istransiting,dicovery,mass,radius,temperature,a,e,I,w,W,period,tt
count,5599.000000,5599.000000,5578.000000,5604.000000,5368.000000,5389.000000,9.00,5380.000000,3373.000000,5363.000000,5378.000000,5384.000000,4434.0,5602.000000,5583.000000,5588.000000,4140.000000,5311.000000,4831.000000,4297.000000,1668.000000,9.000000,5.359000e+03,4.427000e+03
mean,19.585125,15.775684,696.624924,0.950917,1.552338,5430.075238,4.57,12.636057,13.871267,10.991279,10.620030,10.505977,1.0,2016.101214,443.079009,5.690246,910.664857,6.649941,0.076416,86.688931,118.886523,77.732910,7.984349e+04,2.455900e+06
std,35.844526,6.045221,1106.373068,0.423961,4.304881,1776.997636,0.00,3.101634,1.417113,2.983875,3.031439,3.042773,0.0,6.070814,2371.863835,5.302558,460.005472,134.118667,0.150427,11.208090,119.233600,42.291484,5.493977e+06,1.510289e+03
min,-88.121111,0.000000,1.301190,0.010000,0.010000,415.000000,4.57,0.872000,7.713000,-2.095000,-2.775000,-3.044000,1.0,1781.000000,0.002466,0.184252,34.000000,0.004400,0.000000,-0.000543,-233.000000,-5.112604,9.070629e-02,2.440619e+06
25%,-7.826250,12.234444,112.467000,0.790000,0.780000,4945.000000,4.57,10.837500,13.111000,9.096500,8.754000,8.552250,1.0,2014.000000,4.036421,1.782228,568.000000,0.053300,0.000000,86.890000,28.450000,49.713210,4.474399e+00,2.454969e+06
50%,39.883056,19.061944,407.100500,0.950000,0.960000,5557.000000,4.57,13.466000,14.160000,11.891000,11.539000,11.431000,1.0,2016.000000,8.708498,2.779827,819.500000,0.103800,0.000000,88.755000,104.088500,76.672615,1.160650e+01,2.455005e+06
75%,45.434583,19.571944,852.046500,1.090000,1.250000,5900.000000,4.57,15.009000,14.951000,13.376000,12.941000,12.847250,1.0,2020.000000,162.092487,11.789045,1156.250000,0.303150,0.090000,89.670000,210.000000,110.301680,4.175644e+01,2.457083e+06
max,85.736667,23.998333,8500.000000,10.940000,109.460000,57000.000000,4.57,44.610000,19.307700,25.340000,32.340000,33.110000,1.0,2024.000000,89627.610660,77.341967,4050.000000,7506.000000,0.950000,176.092000,395.341000,131.786359,4.020000e+08,2.460640e+06


In [64]:
def filter(dataframe, conditions=[]):
    cond = pd.Series([True]*len(dataframe))
    for condition in conditions:
        column = condition[0]
        value = condition[1]
        if isinstance(value,list):
            cond = (dataframe[column]>=value[0])&(dataframe[column]<=value[1])&cond
        else:
            cond = (dataframe[column] == value)&cond
    
    return dataframe[cond]

In [65]:
filter(exoplanets,[('system_name','ups And')])

,system_name,declination,rightascension,distance,star_name,star_mass,star_radius,star_temperature,star_age,metallicity,...,w,W,M,L,period,et,tt,tp,tk,so
5385,ups And,41.403889,1.613333,13.4054,ups And,1.3,1.56,6156.77,NaN,None,...,324.90,NaN,None,None,4.617033,None,2454083.853,None,None,None
5386,ups And,41.403889,1.613333,13.4054,ups And,1.3,1.56,6156.77,NaN,None,...,241.70,NaN,None,None,241.258000,None,2455133.700,None,None,None
5387,ups And,41.403889,1.613333,13.4054,ups And,1.3,1.56,6156.77,NaN,None,...,258.82,NaN,None,None,1276.460000,None,2453360.600,None,None,None


In [66]:
print_df(filter(exoplanets,[('method','RV'),('e',[0.5,1])]).dropna(subset=['radius']).head(10))

,system_name,declination,rightascension,distance,star_name,star_mass,star_radius,star_temperature,star_age,metallicity,spectraltype,magB,magV,magI,magR,magJ,magH,magK,planet_name,method,istransiting,dicovery,update,mass,radius,temperature,age,b,separation,a,e,I,w,W,M,L,period,et,tt,tp,tk,so
5,16 Cyg B,50.516944,19.697778,21.13970,16 Cyg B,1.08,1.13,5750.00,NaN,None,G3V,None,6.215,NaN,None,4.993,4.695,4.651,16 Cyg B b,RV,1.0,1996.0,97/07/01,565.734564,13.450777,NaN,None,None,None,1.66000,0.680,NaN,82.740,NaN,None,None,798.500000,None,2456937.32,None,None,None
69,BD-17 63,-16.227222,0.000000,34.46030,BD-17 63,0.74,0.69,4714.00,NaN,None,K5V,None,9.620,8.86964,None,7.574,7.027,6.914,BD-17 63 b,RV,NaN,2008.0,09/03/01,1620.924874,12.890328,NaN,None,None,None,1.34000,0.540,NaN,112.200,NaN,None,None,655.600000,None,NaN,None,None,None
78,BD+48 740,48.930000,2.716111,666.58900,BD+48 740,1.09,10.33,4534.00,NaN,None,K3III,None,8.690,NaN,None,6.385,5.791,5.645,BD+48 740 b,RV,NaN,2018.0,18/05/01,540.308291,13.450777,NaN,None,None,None,1.70000,0.760,NaN,100.000,NaN,None,None,733.000000,None,NaN,None,None,None
82,BD+63 1405,63.670000,18.093333,38.04730,BD+63 1405,0.82,0.80,5000.00,NaN,None,K0,None,8.960,NaN,None,7.303,6.895,6.780,BD+63 1405 b,RV,NaN,2021.0,21/07/01,3305.415429,12.441969,NaN,None,None,None,2.06000,0.880,5.974,94.250,NaN,None,None,1198.480000,None,NaN,None,None,None
103,CoRoT-20,0.000000,6.514722,844.06100,CoRoT-20,1.14,1.37,4947.50,NaN,None,G2V,None,14.630,NaN,None,12.991,12.652,12.513,CoRoT-20 c,RV,NaN,2018.0,18/11/01,5403.082912,12.217789,NaN,None,None,None,2.90000,0.600,NaN,65.000,NaN,None,None,1675.000000,None,NaN,None,None,None
200,GJ 1061,-44.514444,3.600278,3.67278,GJ 1061,0.12,0.16,2953.00,NaN,None,M5.5V,None,12.700,NaN,None,7.523,7.015,6.610,GJ 1061 d,RV,NaN,2020.0,20/03/01,1.639995,1.154525,NaN,None,None,None,0.05400,0.530,NaN,157.000,NaN,None,None,13.031000,None,NaN,None,None,None
223,GJ 2056,-24.892222,7.201389,28.44340,GJ 2056,0.62,0.71,4069.95,NaN,None,M0V:,None,10.370,NaN,None,7.704,7.065,6.890,GJ 2056 b,RV,NaN,2020.0,20/10/01,16.199714,4.169741,NaN,None,None,None,0.28300,0.720,NaN,58.000,NaN,None,None,69.971000,None,NaN,None,None,None
230,GJ 3021,-79.851389,0.000000,17.55590,GJ 3021,0.90,0.90,5540.00,NaN,None,G6V,None,6.590,NaN,None,5.366,4.990,4.859,GJ 3021 b,RV,NaN,2000.0,01/08/01,1071.081730,13.114507,350.0,None,None,None,0.49000,0.511,NaN,290.700,NaN,None,None,133.710000,None,NaN,None,None,None
238,GJ 3222,-40.076111,3.393056,18.23817,GJ 3222,0.89,0.93,5257.46,NaN,None,K0V,None,6.900,NaN,None,5.333,4.948,4.767,GJ 3222 b,RV,NaN,2022.0,22/09/01,11.441823,3.396321,NaN,None,None,None,0.09100,0.929,NaN,290.895,NaN,None,None,10.666690,None,NaN,None,None,None
307,GJ 724,-13.382500,18.682500,16.96580,GJ 724,0.53,0.52,3799.00,NaN,None,M1.0V,None,10.638,NaN,None,7.397,6.728,6.546,GJ 724 b,RV,NaN,2023.0,23/10/01,10.748957,3.273022,611.0,None,None,None,0.04685,0.577,NaN,33.200,NaN,None,None,5.101284,None,NaN,None,None,None


In [67]:
systems = pickle.load(open(f"signals/systems-{suffix}.pickle", 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'signals/systems-nasa.pickle'